In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

In [23]:
rb = pd.read_csv('../input/rb/dk_rb_stats.csv')
week1 = pd.read_csv('../input/rb/fantasy-football-weekly-projections (1).csv')
week1_def = pd.read_csv('../input/defense_2018_projections.csv')

In [34]:
rb[rb['Player']=='Ezekiel Elliot']

,Player,Team,Week,Opp,ru_att,ru_yds,yds/att,td,targets,Receptions,rec_yds,rec_td,Fumbles,FumblesLost,points,year,def_ru_rk,def_pass_rk


In [24]:
week1_def.head()

,Rank,Name,Team,Position,Played,TacklesForLoss,Sacks,QuarterbackHits,Interceptions,FumblesRecovered,Safeties,DefensiveTouchdowns,SpecialTeamsTouchdowns,PointsAllowed,FantasyPointsPerGameDraftKings,FantasyPointsDraftKings
0,1,Jacksonville Jaguars,JAX,DST,16,103,49,113,18,14,0,4,1,268,8.1,129
1,2,Los Angeles Chargers,LAC,DST,16,93,44,85,17,13,0,4,2,272,7.9,127
2,3,Los Angeles Rams,LAR,DST,16,82,45,100,17,15,2,3,2,329,7.8,124
3,4,Philadelphia Eagles,PHI,DST,16,99,39,115,17,14,0,4,2,295,7.7,123
4,5,Minnesota Vikings,MIN,DST,16,82,42,85,17,12,1,3,1,252,7.0,112


In [14]:
week1.columns


Index(['Player', 'Team', 'Week', 'Opp', 'ru_att', 'ru_yds', 'yds/att', 'td',
       'targets', 'Receptions', 'rec_yds', 'rec_td', 'Fumbles', 'FumblesLost',
       'points'],
      dtype='object')

## Current week dataframe feature enginerring

In [28]:
week1.drop(['Rank','Position','FantasyPointsPerGame'], axis=1, inplace=True)

week1['year'] =2018
week1.columns = rb.columns[:-2]

indices = []
for idx,player in enumerate(week1['Player']):
    if player not in rb['Player'].unique():
        indices.append(idx)

week1.drop(indices, axis=0, inplace=True)
week1.reset_index(inplace=True, drop=True)

## Weekly Defensive Rankings Dataframe feature engineering

In [30]:
week1_def.index = week1_def['Team']
week1_def.drop('Team', axis=1, inplace=True)
week1_def.head()

week1['def_ru_rk'] = list(map(lambda x: week1_def.loc[x]['Rank'], week1['Opp']))
week1['def_pass_rk'] = list(map(lambda x: week1_def.loc[x]['Rank'], week1['Opp']))


In [31]:
rb.columns

Index(['Player', 'Team', 'Week', 'Opp', 'ru_att', 'ru_yds', 'yds/att', 'td',
       'targets', 'Receptions', 'rec_yds', 'rec_td', 'Fumbles', 'FumblesLost',
       'points', 'year', 'def_ru_rk', 'def_pass_rk'],
      dtype='object')

In [32]:
week1.columns

Index(['Player', 'Team', 'Week', 'Opp', 'ru_att', 'ru_yds', 'yds/att', 'td',
       'targets', 'Receptions', 'rec_yds', 'rec_td', 'Fumbles', 'FumblesLost',
       'points', 'year', 'def_ru_rk', 'def_pass_rk'],
      dtype='object')

In [26]:
#A function that will covert a time series database into a supervised learning database

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
            cols.append(df.shift(i))
            names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [27]:
train  = rb
test = week1_2018

In [29]:
train.shape

(11748, 18)

In [30]:
test.shape

(70, 18)

In [345]:
passing = ['Comp','Att','Pct','Yds','Yds/Att','TD','Int']

In [31]:
y_train = train['points']
y_test = test['points']
X_train = train.drop('points',axis=1)
X_test = test.drop('points', axis=1)

In [32]:
X_train = X_train[X_train.columns[4:]]
X_test = X_test[X_test.columns[4:]]

In [348]:

X_train = X_train[X_train.columns[4:11]]
X_test = X_test[X_test.columns[4:11]]

In [33]:
X_train.columns

Index(['ru_att', 'ru_yds', 'yds/att', 'td', 'targets', 'Receptions', 'rec_yds',
       'rec_td', 'Fumbles', 'FumblesLost', 'year', 'def_ru_rk', 'def_pass_rk'],
      dtype='object')

In [34]:
reframed = series_to_supervised(X_train.values.astype('float32'),1,1)

In [35]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [36]:
coeff = pd.DataFrame(X_train.columns)
coeff['coefficients'] = lr.coef_
coeff



,0,coefficients
0,ru_att,-0.016761
1,ru_yds,0.107790
2,yds/att,-0.020275
3,td,5.971200
4,targets,0.007460
5,Receptions,0.141891
6,rec_yds,0.100108
7,rec_td,5.822190
8,Fumbles,0.010859
9,FumblesLost,-1.777669


In [37]:
yhat = lr.predict(X_test)
yhat.shape

(70,)

In [51]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

rmse_val = rmse(yhat, y_test)
print("RMSE error is: " + str(rmse_val))

RMSE error is: 0.712465875151


In [38]:
results = pd.DataFrame()
results['Player'] = week1_2018['Player']
#results['score'] = y_test
results['Predicted'] = yhat
#results['Week'] = rb[rb['year']==2017]['Week']
#results.groupby('Player').mean()[['score','Predicted']]
results

,Player,Predicted
0,Todd Gurley,18.561469
1,David Johnson,17.240292
2,Melvin Gordon,14.514580
3,Devonta Freeman,11.331842
4,LeSean McCoy,10.591942
5,Lamar Miller,10.344359
6,Le'Veon Bell,9.925670
7,Jay Ajayi,9.253915
8,Rex Burkhead,9.192221
9,Marshawn Lynch,8.668046


In [39]:
results.shape

(70, 2)

In [40]:
results.to_csv('2018_week1_rb.csv', index= False)